In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch.nn.functional as F
import torch
from torch import nn
from torchsummary import summary
from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [2]:
from omegaconf import OmegaConf

In [ ]:
path = "../training/configs/vqvae.yaml"

In [ ]:
conf = OmegaConf.load(path)

In [ ]:
print(OmegaConf.to_yaml(conf))

In [ ]:
from text_recognizer.networks import VQVAE

In [ ]:
vae = VQVAE(**conf.network.args)

In [ ]:
vae

In [ ]:
datum = torch.randn([2, 1, 576, 640])

In [ ]:
vae.encoder(datum)[0].shape

In [ ]:
vae(datum)[0].shape

In [ ]:
from text_recognizer.networks.backbones.efficientnet import EfficientNet

In [ ]:
en = EfficientNet()

In [3]:
datum = torch.randn([2, 1, 576, 640])

In [4]:
trg = torch.randint(0, 1000, [2, 682])

In [ ]:
trg.shape

In [ ]:
datum = torch.randn([2, 1, 224, 224])

In [ ]:
en(datum).shape

In [5]:
path = "../training/configs/cnn_transformer.yaml"

In [6]:
conf = OmegaConf.load(path)

In [7]:
print(OmegaConf.to_yaml(conf))

seed: 4711
network:
  desc: Configuration of the PyTorch neural network.
  type: CNNTransformer
  args:
    encoder:
      type: EfficientNet
      args: null
    num_decoder_layers: 4
    hidden_dim: 256
    num_heads: 4
    expansion_dim: 1024
    dropout_rate: 0.1
    transformer_activation: glu
model:
  desc: Configuration of the PyTorch Lightning model.
  type: LitTransformerModel
  args:
    optimizer:
      type: MADGRAD
      args:
        lr: 0.001
        momentum: 0.9
        weight_decay: 0
        eps: 1.0e-06
    lr_scheduler:
      type: OneCycleLR
      args:
        interval: step
        max_lr: 0.001
        three_phase: true
        epochs: 512
        steps_per_epoch: 1246
    criterion:
      type: CrossEntropyLoss
      args:
        weight: None
        ignore_index: -100
        reduction: mean
    monitor: val_loss
    mapping: sentence_piece
data:
  desc: Configuration of the training/test data.
  type: IAMExtendedParagraphs
  args:
    batch_size: 16
    num

In [8]:
from text_recognizer.networks.cnn_transformer import CNNTransformer

In [9]:
t = CNNTransformer(input_shape=(1, 576, 640), output_shape=(682, 1), **conf.network.args)

In [ ]:
t.encode(datum).shape

In [ ]:
trg.shape

In [10]:
t(datum, trg).shape

torch.Size([2, 682, 1004])

In [7]:
b, n = 16, 128
device = "cpu"

In [8]:
x = lambda: torch.ones((b, n), device=device).bool()

In [10]:
x().shape

torch.Size([16, 128])

In [12]:
torch.ones((b, n), device=device).bool().shape

torch.Size([16, 128])

In [24]:
x = torch.randn(1, 1, 576, 640)

In [30]:
576 // 4

144

In [29]:
640 // 4

160

In [31]:
x = torch.randn(1, 1, 144, 160)

In [32]:
from einops import rearrange

In [35]:
patch_size=4
p = rearrange(x, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_size, p2 = patch_size)

In [36]:
p.shape

torch.Size([1, 1440, 16])